In [180]:
'''
Created on 2016. 12. 10

@author: dato
@desc: cnlp2 hw8 (EM clustering)
'''
#-*- coding: utf-8 -*-

'\nCreated on 2016. 12. 10\n\n@author: dato\n@desc: cnlp2 hw8 (EM clustering)\n'

In [181]:
import numpy as np
import math

In [16]:
f = open("emcluster_sample.txt")
data = f.readlines()

In [17]:
# probability table for each class
prob_1 = {}
prob_2 = {}

In [18]:
def dic_log_norm(dic):
    new_dic = {}
    s = sum(dic.values())    
    for item in dic:        
        new_dic[item] = math.log(dic[item] / float(s))
    return new_dic

In [43]:
def cal_prob_table(data, label):

    prob_1 = {}
    prob_2 = {}
    
    for line in data:

        line = line.strip().split(' ')
        doc_id = int(line[0])
        
        if( prob_1.has_key( line[1]) ):
                prob_1[ line[1] ] = prob_1[ line[1] ] + float(line[2]) * label[ doc_id -1 ]
        else:
                prob_1[ line[1] ] = float(line[2]) * label[ doc_id -1 ]

        if( prob_2.has_key(line[1])):
                prob_2[ line[1] ] = prob_2[ line[1] ] + float(line[2]) * (1 - label[ doc_id -1 ])
        else:
                prob_2[ line[1] ] = float(line[2]) * (1 - label[ doc_id -1 ])
    
    return dic_log_norm(prob_1), dic_log_norm(prob_2)

In [119]:
def update_label(data, prob_1, prob_2, prior_1, prior_2):

    cnt = 0
    
    sum1 = np.zeros(980, dtype=np.float64)
    sum2 = np.zeros(980, dtype=np.float64)
    label = np.zeros(980, dtype=np.float64)
    
    for line in data:
        
        line = line.strip().split(' ')
        doc_id = int(line[0])

        # add-up log P(v/c)
        sum1[doc_id -1] = sum1[doc_id -1] + prob_1[ line[1] ] * float(line[2])
        sum2[doc_id -1] = sum2[doc_id -1] + prob_2[ line[1] ] * float(line[2])

    # add log P(c)
    for i in range(doc_id):
        final1 = sum1[i -1] + prior_1
        final2 = sum2[i -1] + prior_2    
        label[i - 1] = final1 / float(final1 + final2)

    return label

In [120]:
def cal_ll(data, prob_1, prob_2, prior_1, prior_2):

    cnt = 0
    sum1 = 0
    sum2 = 0
    index = 1
    final1 = 0
    final2 = 0
    
    ll = 0
    
    label = np.ndarray(980, dtype=np.float64)
    
    for line in data:

        line = line.strip().split(' ')

        # same doc_id
        # add-up log P(v/c)
        if ( int(line[0]) == index ):
            sum1 = sum1 + prob_1[ line[1] ] * float(line[2])
            sum2 = sum2 + prob_2[ line[1] ] * float(line[2])

        # doc_id is changed
        else:

            # add log P(c)
            final1 = sum1 + prior_1
            final2 = sum2 + prior_2
            
            ll = (final1 * 0.5) + (final2 * 0.5)
                        
            sum1 = 0
            sum2 = 0
            sum1 = sum1 + prob_1[ line[1] ] * float(line[2])
            sum2 = sum2 + prob_2[ line[1] ] * float(line[2])
            
            index = index + 1

    return ll

In [172]:
prob_1['1'] = math.log(0.25)
prob_1['2'] = math.log(0.25)
prob_1['3'] = math.log(0.25)
prob_1['4'] = math.log(0.25)

prob_2['1'] = math.log(0.25)
prob_2['2'] = math.log(0.25)
prob_2['3'] = math.log(0.25)
prob_2['4'] = math.log(0.25)

prior_1 = math.log(0.6)
prior_2 = math.log(0.4)

ll = np.zeros(500, dtype=np.float64)
h = np.zeros(500, dtype=np.float64)

In [173]:
for loop_index in range (500):

    # E step    
    label = update_label(data, prob_1, prob_2, prior_1, prior_2)
    h[loop_index -1] = np.mean(label)    
    
    # M step
    prob_1_new, prob_2_new = cal_prob_table(data, label)    
    prior_1  = sum(label) / len(label)
    prior_2 = 1 - prior_1
    
    ll[loop_index -1] = cal_ll(data, prob_1_new, prob_2_new, prior_1, prior_2)
    
    cond_sum = 0
    for i in range(4):
        cond_sum = cond_sum + 1 - prob_1[str(i+1)] / prob_1_new[str(i+1)] 
            
    if ( abs(cond_sum) < 1e-5 ):
        break
        
    else :
        prob_1 = prob_1_new
        prob_2 = prob_2_new       

In [174]:
print loop_index

5


In [175]:
# log P(c)

print prior_1
print prior_2
print prob_1
print prob_2

0.500298111485
0.499701888515
{'1': -1.3164000545081527, '3': -1.3056135886078815, '2': -1.3224928862875682, '4': -1.6377093018372164}
{'1': -1.3185593589247135, '3': -1.3015482917100176, '2': -1.32450264353169, '4': -1.6376613581997104}


In [176]:
# log P(v|c)

print math.exp(prob_1['1'])
print math.exp(prob_1['2'])
print math.exp(prob_1['3'])
print math.exp(prob_1['4'])

print math.exp(prob_2['1'])
print math.exp(prob_2['2'])
print math.exp(prob_2['3'])
print math.exp(prob_2['4'])

0.268098707555
0.266470193399
0.271006197704
0.194424901342
0.2675204254
0.26593519079
0.272110160807
0.194434223002


In [177]:
# responsibility P(c|d)

print label[:loop_index]

[ 0.49981446  0.50039636  0.50055607  0.50029989  0.50167441]


In [178]:
# loglikelihood

print ll[:loop_index]*-1

[ 0.81746267  0.81747198  0.81747797  0.81747971  0.81748005]
